In [1]:
import numpy as np
import os
import tensorflow as tf
import warnings

from keras.models import Model
from keras.layers import Dense, Flatten, Convolution2D, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from pommerman.configs import ffa_v0_env
from pommerman.envs.v0 import Pomme
from pommerman.agents import SimpleAgent, BaseAgent
from pommerman.constants import BOARD_SIZE
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.core import Env, Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint, Callback

/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
number_of_training_steps = 5000000
log_interval = 1000
file_log_path = './dqn/rl_logs/dense_8_2/log.txt'
tensorboard_path = './dqn/logs/dense_8_2/'
model_path = './dqn/model/dense_8_2/model{step}.h4'

In [3]:
if not os.path.isdir(os.path.dirname(file_log_path)):
    os.makedirs(os.path.dirname(file_log_path))
if not os.path.isdir(os.path.dirname(model_path)):
    os.makedirs(os.path.dirname(model_path))

In [9]:
class TensorforceAgent(BaseAgent):
    def act(self, obs, action_space):
        pass


class TensorboardLogger(Callback):
    """Logging in tensorboard without tensorflow ops."""
    def __init__(self, log_dir):
        # Some algorithms compute multiple episodes at once since they are multi-threaded.
        # We therefore use a dictionary that is indexed by the episode to separate episodes
        # from each other.
        self.observations = {}
        self.rewards = {}
        self.actions = {}
        self.metrics = {}
        self.step = 0
        """Creates a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def log_scalar(self, tag, value, step):
        """Log a scalar variable.
        Parameter
        ----------
        tag : basestring
            Name of the scalar
        value
        step : int
            training iteration
        """
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def on_train_begin(self, logs):
        self.metrics_names = self.model.metrics_names

    def on_episode_begin(self, episode, logs):
        self.observations[episode] = []
        self.rewards[episode] = []
        self.actions[episode] = []
        self.metrics[episode] = []

    def on_episode_end(self, episode, logs):
        episode_steps = len(self.observations[episode])
        variables = {
            'step': self.step,
            'nb_steps': self.params['nb_steps'],
            'episode_steps': episode_steps,
            'episode_reward': np.sum(self.rewards[episode]),
            'reward_mean': np.mean(self.rewards[episode]),
            'reward_min': np.min(self.rewards[episode]),
            'reward_max': np.max(self.rewards[episode]),
            'action_mean': np.mean(self.actions[episode]),
            'action_min': np.min(self.actions[episode]),
            'action_max': np.max(self.actions[episode]),
            'obs_mean': np.mean(self.observations[episode]),
            'obs_min': np.min(self.observations[episode]),
            'obs_max': np.max(self.observations[episode]),
        }

        # Format all metrics.
        metrics = np.array(self.metrics[episode])
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            for idx, name in enumerate(self.metrics_names):
                try:
                    value = np.nanmean(metrics[:, idx])
                except Warning:
                    value = -1
                variables[name] = value
        for key, value in variables.items():
            self.log_scalar(key, value, episode + 1)

        # Free up resources.
        del self.observations[episode]
        del self.rewards[episode]
        del self.actions[episode]
        del self.metrics[episode]

    def on_step_end(self, step, logs):
        episode = logs['episode']
        self.observations[episode].append(logs['observation'])
        self.rewards[episode].append(logs['reward'])
        self.actions[episode].append(logs['action'])
        self.metrics[episode].append(logs['metrics'])
        self.step += 1

In [10]:
# Instantiate the environment
config = ffa_v0_env()
env = Pomme(**config["env_kwargs"])
np.random.seed(0)
env.seed(0)
# Add 3 random agents
agents = []
for agent_id in range(3):
    agents.append(SimpleAgent(config["agent"](agent_id, config["game_type"])))

# Add TensorforceAgent
agent_id += 1
agents.append(TensorforceAgent(config["agent"](agent_id, config["game_type"])))
env.set_agents(agents)
env.set_training_agent(agents[-1].agent_id)
env.set_init_game_state(None)
nb_actions = env.action_space.n


def create_model(actions, input_shape=(2369,)):
    inp = Input(input_shape)        
    x = Dense(8, activation='relu')(inp)
    x = Dense(8, activation='relu')(x)
    out = Dense(actions)(x)
    model = Model(inputs = inp, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Next, we build a very simple model regardless of the dueling architecture
# if you enable dueling network in DQN , DQN will build a dueling network base on your model automatically
# Also, you can build a dueling network by yourself and turn off the dueling network in DQN.
model = create_model(nb_actions)
print(model.summary())

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2369)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 18960     
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 54        
Total params: 19,086
Trainable params: 19,086
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
class EnvWrapper(Env):
    """The abstract environment class that is used by all agents. This class has the exact
        same API that OpenAI Gym uses so that integrating with it is trivial. In contrast to the
        OpenAI Gym implementation, this class only defines the abstract methods without any actual
        implementation.
        To implement your own environment, you need to define the following methods:
        - `step`
        - `reset`
        - `render`
        - `close`
        Refer to the [Gym documentation](https://gym.openai.com/docs/#environments).
        """
    reward_range = (-1, 1)
    action_space = None
    observation_space = None

    def __init__(self, gym, board_size):
        self.gym = gym
        self.action_space = gym.action_space
        self.observation_space = gym.observation_space
        self.reward_range = gym.reward_range
        self.board_size = board_size

    def step(self, action):
        """Run one timestep of the environment's dynamics.
        Accepts an action and returns a tuple (observation, reward, done, info).
        # Arguments
            action (object): An action provided by the environment.
        # Returns
            observation (object): Agent's observation of the current environment.
            reward (float) : Amount of reward returned after previous action.
            done (boolean): Whether the episode has ended, in which case further step() calls will return undefined results.
            info (dict): Contains auxiliary diagnostic information (helpful for debugging, and sometimes learning).
        """
        obs = self.gym.get_observations()
        all_actions = self.gym.act(obs)
        all_actions.insert(self.gym.training_agent, action)
        state, reward, terminal, info = self.gym.step(all_actions)
        agent_state = self.featurize(state[self.gym.training_agent])
        agent_reward = reward[self.gym.training_agent]
        return agent_state, agent_reward, terminal, info

    def reset(self):
        """
        Resets the state of the environment and returns an initial observation.
        # Returns
            observation (object): The initial observation of the space. Initial reward is assumed to be 0.
        """
        obs = self.gym.reset()
        agent_obs = self.featurize(obs[self.gym.training_agent])
        return agent_obs

    def render(self, mode='human', close=False):
        """Renders the environment.
        The set of supported modes varies per environment. (And some
        environments do not support rendering at all.)
        # Arguments
            mode (str): The mode to render with.
            close (bool): Close all open renderings.
        """
        self.gym.render(mode=mode, close=close)

    def close(self):
        """Override in your subclass to perform any necessary cleanup.
        Environments will automatically close() themselves when
        garbage collected or when the program exits.
        """
        self.gym.close()

    def seed(self, seed=None):
        """Sets the seed for this env's random number generator(s).
        # Returns
            Returns the list of seeds used in this env's random number generators
        """
        raise self.gym.seed(seed)

    def configure(self, *args, **kwargs):
        """Provides runtime configuration to the environment.
        This configuration should consist of data that tells your
        environment how to run (such as an address of a remote server,
        or path to your ImageNet data). It should not affect the
        semantics of the environment.
        """
        raise NotImplementedError()

    def featurize(self, obs):
        shape = (self.board_size, self.board_size, 1)

        def get_matrix(dict, key):
            res = dict[key]
            return res.reshape(shape).astype(np.float32)

        def get_map(board, item):
            map = np.zeros(shape)
            map[board == item] = 1
            return map

        board = get_matrix(obs, 'board')

        # TODO: probably not needed Passage = 0
        rigid_map = get_map(board, 1)               # Rigid = 1
        wood_map = get_map(board, 2)                # Wood = 2
        bomb_map = get_map(board, 3)                # Bomb = 3
        flames_map = get_map(board, 4)              # Flames = 4
        fog_map = get_map(board, 5)                 # TODO: not used for first two stages Fog = 5
        extra_bomb_map = get_map(board, 6)          # ExtraBomb = 6
        incr_range_map = get_map(board, 7)          # IncrRange = 7
        kick_map = get_map(board, 8)                # Kick = 8
        skull_map = get_map(board, 9)               # Skull = 9

        position = obs["position"]
        my_position = np.zeros(shape)
        my_position[position[0], position[1], 0] = 1

        team_mates = get_map(board, obs["teammate"].value) # TODO during documentation it should be an array

        enemies = np.zeros(shape)
        for enemy in obs["enemies"]:
            enemies[board == enemy.value] = 1

        bomb_blast_strength = get_matrix(obs, 'bomb_blast_strength')
        bomb_life = get_matrix(obs, 'bomb_life')

        ammo = obs["ammo"]
        blast_strength = obs["blast_strength"]
        can_kick = int(obs["can_kick"])

        obs = np.concatenate([my_position, enemies, team_mates, rigid_map,
                              wood_map, bomb_map, flames_map,
                              fog_map, extra_bomb_map, incr_range_map,
                              kick_map, skull_map, bomb_blast_strength,
                              bomb_life], axis=2).flatten()
        obs = np.append(obs, [ammo, blast_strength, can_kick])
        return obs

    def __del__(self):
        self.close()

    def __str__(self):
        return '<{} instance>'.format(type(self).__name__)


class CustomProcessor(Processor):
    def process_state_batch(self, batch):
        """Processes an entire batch of states and returns it.
        # Arguments
            batch (list): List of states
        # Returns
            Processed list of states
        """
        batch = np.squeeze(batch, axis=1)
        return batch

    def process_info(self, info):
        """Processes the info as obtained from the environment for use in an agent and
        returns it.
        """
        info['result'] = info['result'].value
        return info

In [12]:
env_wrapper = EnvWrapper(env, BOARD_SIZE)
processor = CustomProcessor()

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
file_logger = FileLogger(file_log_path, interval=log_interval)
checkpoint = ModelIntervalCheckpoint(model_path, interval=log_interval)
tensorboard = TensorboardLogger(tensorboard_path)
reduce_rl = ReduceLROnPlateau(monitor='loss', factor=0.9, patience=3)
callbacks=[file_logger, checkpoint, tensorboard]
# enable the dueling network
# you can specify the dueling_type to one of {'avg','max','naive'}
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=512,
               enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy,
               processor=processor, batch_size=512)
dqn.compile(Adam(lr=1e-4), metrics=['mae'])
if os.path.isfile(model_path):
    dqn.load_weights(model_path)

In [13]:
history = dqn.fit(env_wrapper, nb_steps=number_of_training_steps, visualize=False, verbose=2,
        nb_max_episode_steps=env._max_steps,
                  callbacks=callbacks)

Training for 5000000 steps ...
      33/5000000: episode: 1, duration: 0.518s, episode steps: 33, steps per second: 64, episode reward: -1.000, mean reward: -0.030 [-1.000, 0.000], mean action: 2.273 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: --, mean_absolute_error: --, mean_q: --
     125/5000000: episode: 2, duration: 1.386s, episode steps: 92, steps per second: 66, episode reward: -1.000, mean reward: -0.011 [-1.000, 0.000], mean action: 2.772 [0.000, 5.000], mean observation: 0.072 [0.000, 24.000], loss: --, mean_absolute_error: --, mean_q: --
     185/5000000: episode: 3, duration: 0.926s, episode steps: 60, steps per second: 65, episode reward: -1.000, mean reward: -0.017 [-1.000, 0.000], mean action: 2.367 [0.000, 5.000], mean observation: 0.072 [0.000, 24.000], loss: --, mean_absolute_error: --, mean_q: --
     212/5000000: episode: 4, duration: 0.514s, episode steps: 27, steps per second: 52, episode reward: -1.000, mean reward: -0.037 [-1.000, 0.000], mea

    1446/5000000: episode: 30, duration: 1.692s, episode steps: 35, steps per second: 21, episode reward: -1.000, mean reward: -0.029 [-1.000, 0.000], mean action: 2.029 [0.000, 5.000], mean observation: 0.066 [0.000, 24.000], loss: 0.000702, mean_absolute_error: 0.460401, mean_q: -0.538457
    1477/5000000: episode: 31, duration: 1.485s, episode steps: 31, steps per second: 21, episode reward: -1.000, mean reward: -0.032 [-1.000, 0.000], mean action: 2.742 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.000697, mean_absolute_error: 0.460561, mean_q: -0.539018
    1592/5000000: episode: 32, duration: 5.345s, episode steps: 115, steps per second: 22, episode reward: -1.000, mean reward: -0.009 [-1.000, 0.000], mean action: 2.374 [0.000, 5.000], mean observation: 0.069 [0.000, 24.000], loss: 0.000663, mean_absolute_error: 0.460218, mean_q: -0.538551
    1755/5000000: episode: 33, duration: 7.828s, episode steps: 163, steps per second: 21, episode reward: -1.000, mean rew

    3145/5000000: episode: 59, duration: 1.495s, episode steps: 31, steps per second: 21, episode reward: -1.000, mean reward: -0.032 [-1.000, 0.000], mean action: 2.484 [0.000, 5.000], mean observation: 0.071 [0.000, 24.000], loss: 0.000821, mean_absolute_error: 0.447495, mean_q: -0.521802
    3171/5000000: episode: 60, duration: 1.282s, episode steps: 26, steps per second: 20, episode reward: -1.000, mean reward: -0.038 [-1.000, 0.000], mean action: 2.346 [0.000, 5.000], mean observation: 0.071 [0.000, 24.000], loss: 0.000813, mean_absolute_error: 0.448672, mean_q: -0.523429
    3200/5000000: episode: 61, duration: 1.394s, episode steps: 29, steps per second: 21, episode reward: -1.000, mean reward: -0.034 [-1.000, 0.000], mean action: 2.172 [0.000, 5.000], mean observation: 0.068 [0.000, 24.000], loss: 0.000787, mean_absolute_error: 0.448954, mean_q: -0.523448
    3225/5000000: episode: 62, duration: 1.266s, episode steps: 25, steps per second: 20, episode reward: -1.000, mean rewar

    4938/5000000: episode: 88, duration: 1.587s, episode steps: 32, steps per second: 20, episode reward: -1.000, mean reward: -0.031 [-1.000, 0.000], mean action: 2.875 [0.000, 5.000], mean observation: 0.071 [0.000, 24.000], loss: 0.000907, mean_absolute_error: 0.455043, mean_q: -0.529358
    4992/5000000: episode: 89, duration: 2.633s, episode steps: 54, steps per second: 21, episode reward: -1.000, mean reward: -0.019 [-1.000, 0.000], mean action: 3.074 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.000920, mean_absolute_error: 0.454852, mean_q: -0.529004
    5025/5000000: episode: 90, duration: 2.586s, episode steps: 33, steps per second: 13, episode reward: -1.000, mean reward: -0.030 [-1.000, 0.000], mean action: 2.515 [0.000, 5.000], mean observation: 0.071 [0.000, 24.000], loss: 0.000934, mean_absolute_error: 0.451426, mean_q: -0.524531
    5221/5000000: episode: 91, duration: 9.613s, episode steps: 196, steps per second: 20, episode reward: -1.000, mean rewa

    6371/5000000: episode: 116, duration: 5.701s, episode steps: 124, steps per second: 22, episode reward: -1.000, mean reward: -0.008 [-1.000, 0.000], mean action: 2.331 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.000850, mean_absolute_error: 0.435202, mean_q: -0.507041
    6435/5000000: episode: 117, duration: 3.125s, episode steps: 64, steps per second: 20, episode reward: -1.000, mean reward: -0.016 [-1.000, 0.000], mean action: 2.000 [0.000, 5.000], mean observation: 0.066 [0.000, 24.000], loss: 0.000880, mean_absolute_error: 0.435194, mean_q: -0.507706
    6461/5000000: episode: 118, duration: 1.305s, episode steps: 26, steps per second: 20, episode reward: -1.000, mean reward: -0.038 [-1.000, 0.000], mean action: 2.808 [0.000, 5.000], mean observation: 0.071 [0.000, 24.000], loss: 0.000947, mean_absolute_error: 0.437327, mean_q: -0.509450
    6490/5000000: episode: 119, duration: 1.471s, episode steps: 29, steps per second: 20, episode reward: -1.000, mean 

...

  657968/5000000: episode: 10941, duration: 7.117s, episode steps: 140, steps per second: 20, episode reward: -1.000, mean reward: -0.007 [-1.000, 0.000], mean action: 2.593 [0.000, 5.000], mean observation: 0.069 [0.000, 24.000], loss: 0.028312, mean_absolute_error: 0.828851, mean_q: 0.706918
  658003/5000000: episode: 10942, duration: 1.695s, episode steps: 35, steps per second: 21, episode reward: -1.000, mean reward: -0.029 [-1.000, 0.000], mean action: 2.457 [0.000, 5.000], mean observation: 0.065 [0.000, 24.000], loss: 0.029997, mean_absolute_error: 0.835643, mean_q: 0.694828
  658128/5000000: episode: 10943, duration: 5.882s, episode steps: 125, steps per second: 21, episode reward: -1.000, mean reward: -0.008 [-1.000, 0.000], mean action: 2.456 [0.000, 5.000], mean observation: 0.072 [0.000, 24.000], loss: 0.028174, mean_absolute_error: 0.828727, mean_q: 0.702303
  658153/5000000: episode: 10944, duration: 1.201s, episode steps: 25, steps per second: 21, episode reward: -1.000,

  659559/5000000: episode: 10969, duration: 4.853s, episode steps: 102, steps per second: 21, episode reward: -1.000, mean reward: -0.010 [-1.000, 0.000], mean action: 2.706 [0.000, 5.000], mean observation: 0.071 [0.000, 24.000], loss: 0.026779, mean_absolute_error: 0.780810, mean_q: 0.636011
  659638/5000000: episode: 10970, duration: 3.424s, episode steps: 79, steps per second: 23, episode reward: -1.000, mean reward: -0.013 [-1.000, 0.000], mean action: 2.658 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.026693, mean_absolute_error: 0.781114, mean_q: 0.630455
  659696/5000000: episode: 10971, duration: 2.899s, episode steps: 58, steps per second: 20, episode reward: -1.000, mean reward: -0.017 [-1.000, 0.000], mean action: 2.569 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.026271, mean_absolute_error: 0.778446, mean_q: 0.636032
  659793/5000000: episode: 10972, duration: 4.968s, episode steps: 97, steps per second: 20, episode reward: -1.000, 

  661218/5000000: episode: 10997, duration: 1.843s, episode steps: 37, steps per second: 20, episode reward: -1.000, mean reward: -0.027 [-1.000, 0.000], mean action: 2.351 [0.000, 5.000], mean observation: 0.069 [0.000, 24.000], loss: 0.022172, mean_absolute_error: 0.739641, mean_q: 0.561902
  661245/5000000: episode: 10998, duration: 1.292s, episode steps: 27, steps per second: 21, episode reward: -1.000, mean reward: -0.037 [-1.000, 0.000], mean action: 2.111 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.021555, mean_absolute_error: 0.729872, mean_q: 0.556713
  661348/5000000: episode: 10999, duration: 4.947s, episode steps: 103, steps per second: 21, episode reward: -1.000, mean reward: -0.010 [-1.000, 0.000], mean action: 2.524 [0.000, 5.000], mean observation: 0.072 [0.000, 24.000], loss: 0.022541, mean_absolute_error: 0.725504, mean_q: 0.553397
  661373/5000000: episode: 11000, duration: 1.329s, episode steps: 25, steps per second: 19, episode reward: -1.000, 

  662551/5000000: episode: 11025, duration: 3.001s, episode steps: 57, steps per second: 19, episode reward: -1.000, mean reward: -0.018 [-1.000, 0.000], mean action: 2.491 [0.000, 5.000], mean observation: 0.069 [0.000, 24.000], loss: 0.022117, mean_absolute_error: 0.700569, mean_q: 0.510819
  662579/5000000: episode: 11026, duration: 1.343s, episode steps: 28, steps per second: 21, episode reward: -1.000, mean reward: -0.036 [-1.000, 0.000], mean action: 2.321 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.021739, mean_absolute_error: 0.709593, mean_q: 0.503329
  662840/5000000: episode: 11027, duration: 11.943s, episode steps: 261, steps per second: 22, episode reward: -1.000, mean reward: -0.004 [-1.000, 0.000], mean action: 2.464 [0.000, 5.000], mean observation: 0.060 [0.000, 24.000], loss: 0.021771, mean_absolute_error: 0.697419, mean_q: 0.494710
  662867/5000000: episode: 11028, duration: 1.306s, episode steps: 27, steps per second: 21, episode reward: -1.000,

  663975/5000000: episode: 11053, duration: 1.564s, episode steps: 31, steps per second: 20, episode reward: -1.000, mean reward: -0.032 [-1.000, 0.000], mean action: 2.581 [0.000, 5.000], mean observation: 0.070 [0.000, 24.000], loss: 0.022474, mean_absolute_error: 0.668428, mean_q: 0.451193
  664089/5000000: episode: 11054, duration: 5.562s, episode steps: 114, steps per second: 20, episode reward: -1.000, mean reward: -0.009 [-1.000, 0.000], mean action: 2.553 [0.000, 5.000], mean observation: 0.066 [0.000, 24.000], loss: 0.020968, mean_absolute_error: 0.669187, mean_q: 0.448186
  664114/5000000: episode: 11055, duration: 1.261s, episode steps: 25, steps per second: 20, episode reward: -1.000, mean reward: -0.040 [-1.000, 0.000], mean action: 2.760 [0.000, 5.000], mean observation: 0.072 [0.000, 24.000], loss: 0.022771, mean_absolute_error: 0.663457, mean_q: 0.449965
  664139/5000000: episode: 11056, duration: 1.191s, episode steps: 25, steps per second: 21, episode reward: -1.000, 

In [14]:
dqn.save_weights(model_path, overwrite=True)